In [ ]:
#!pip install --quiet playwright
#!playwright install

We're going to use [Playwright](https://playwright.dev/python/) to gain access to the content on TikTok.

But instead of writing some magic custom scraper to automatically do stuff, we're going to just use... ourselves! While we browse, though, we're going to watch everything that the browser does and save all of the parts we think are interesting.

# Open up our browser

TikTik *does know* we're kind of a bot, even though we're using `--disable-blink-features=AutomationControlled`. I'm not sure how to fix it, but it's fine for now because they don't block you, just make you unable to watch videos (kind of).

But here we go!

In [105]:
from playwright.async_api import async_playwright

args = [
    "--disable-blink-features=AutomationControlled"
]
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False, args=args)
page = await browser.new_page()

## Add a listener for API events

When we visit a page, behind the scenes our browser is having a lot of conversations with the website. We're going to add a listener that looks for those conversations, and intercepts any of them that start with `https://www.tiktok.com/api/`.

When it finds one, it saves the details to the `api` folder.

In [4]:
from urllib.parse import parse_qs, urlparse
import hashlib
from pathlib import Path
import json

In [106]:
from urllib.parse import parse_qs, urlparse
import hashlib
from pathlib import Path
import json

# If we've already added the listener, remove it
try:
    page.remove_listener('response', process_response)
except:
    pass

async def process_response(response):
    if response.ok and response.url.startswith("https://www.tiktok.com/api/"):
        try:
            parsed = urlparse(response.url)

            # Hash the query parameters and the response.text to make a unique filename
            m = hashlib.md5()
            m.update(parsed.query.encode('utf-8'))
            m.update((await response.text()).encode('utf-8'))
            filename = m.hexdigest()

            # the full path is based on the URL
            path = Path(parsed.path.lstrip("/")).joinpath(filename).with_suffix(".json")
            path.parent.mkdir(parents=True, exist_ok=True)

            print("Writing content to ", path)

            query = parse_qs(parsed.query)
            query = {k: v[0] for k, v in query.items()}

            content = {
                'url': response.url,
                'query': query,
                'headers': await response.all_headers(),
                'data': await response.json(),
            }

            path.write_text(json.dumps(content, indent=2))
        except Exception as e:
            print("Error processing response", e)

page.on("response", process_response)

Writing content to  api/policy/notice/a955a349b6123ee5fc9b4faac150b944.json
Writing content to  api/post/item_list/a293da03fdd8a5296605c93ad77fe5de.json
Writing content to  api/ba/business/suite/permission/list/52dceb30db7d1aca5f44f4f6149d1d3f.json
Error processing response Expecting value: line 1 column 1 (char 0)
Writing content to  api/user/playlist/7b5a01e70de3fd6184f2b6c6b779f685.json
Writing content to  api/share/settings/fea309bf56c06ef4df9ed619a74f12ad.json
Writing content to  api/repost/item_list/38c8dd1465255bf091c5bd672f36177c.json
Writing content to  api/post/item_list/0ebce89ffbda59a0f72dffa0c9a4e990.json
Writing content to  api/preload/item_list/ce8971a2dba9ff2cd4a5048537fc663c.json
Writing content to  api/post/item_list/0b0462f8167de37043486bf1b58ef4eb.json
Writing content to  api/post/item_list/5acdf190d6edfd9d0b0f47edcbc7f95a.json
Writing content to  api/post/item_list/3853649a5c1c9d21e724037c72329648.json
Writing content to  api/post/item_list/ae01cfc084dcb55c54d763ae

In [107]:
await browser.close()

## Look at the videos

We have a lot of different things that are saved - comments, related items, etc – but the videos are what we're interested in here. The requests for the videos are saved in `api/post/item_list`.

Obviously enough, each one is a list of posts. Let's look at a single one of the JSON files.

In [59]:
import glob

# Just grab the first one
filename = glob.glob("api/post/item_list/*.json")[-1]

In [40]:
from pprint import pprint

with open(filename) as fp:
    content = json.load(fp)

content.keys()

dict_keys(['url', 'query', 'headers', 'data'])

Each response has an `itemList`, which is a list videos. We can look at the first one:

In [20]:
pprint(content['data']['itemList'][0])

{'AIGCDescription': '',
 'author': {'avatarLarger': 'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/5a8bae33b06523b66547e64d11a27155~c5_1080x1080.jpeg?lk3s=a5d48078&nonce=57550&refresh_token=a346a60f2470f8bb8575cf1f979a6a2a&x-expires=1718982000&x-signature=F8XSEmwE0fTDATz3g3OR0EUWyOg%3D&shp=a5d48078&shcp=81f88b70',
            'avatarMedium': 'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/5a8bae33b06523b66547e64d11a27155~c5_720x720.jpeg?lk3s=a5d48078&nonce=49451&refresh_token=addb7823f04c31f5807e628c486303b6&x-expires=1718982000&x-signature=6fOptkg1DAbU5edvs6%2BqkHjVDGg%3D&shp=a5d48078&shcp=81f88b70',
            'avatarThumb': 'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/5a8bae33b06523b66547e64d11a27155~c5_100x100.jpeg?lk3s=a5d48078&nonce=85009&refresh_token=ad1c7855c1b61c44869301a26676651c&x-expires=1718982000&x-signature=NqUanteJOsIyvI2e%2FKg8RztE%2BRk%3D&shp=a5d48078&shcp=81f88b70',
            'commentSetting': 0,
            'downloadSetting': 0,
        

## Get all videos

We're hungry for more, so let's grab all of the JSON files that contain video data and combine them into one big pandas dataframe.

In [108]:
import glob

filenames = glob.glob("api/post/item_list/*.json")
filenames[:5]

['api/post/item_list/5cb0c15c4fbe664ad4c272d68b8590e3.json',
 'api/post/item_list/6b45c2be1e07e9a5680bc5341fe54725.json',
 'api/post/item_list/3a215d61d9bf7dc60d7fe9622380a31b.json',
 'api/post/item_list/7ef63e219a428d8192393c3d2bf07266.json',
 'api/post/item_list/348b654d28f07521d668432862694ead.json']

In [109]:
rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['itemList'])

len(rows)

5556

The data in the JSON response is nested in a wild way, so we'll use `pd.json_normalize` to "flatten" the dictionaries inside of dictionaries inside of dictionaries into something simpler.

In [110]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 75

# Use json_normalize bc see above
videos_df = pd.json_normalize(rows, sep='_')

videos_df.head(2)

,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'How to make a handmade bow! #wrappinggifts #wrappingpresents...,1637693876,How to make a handmade bow! #wrappinggifts #wrappingpresents #giftwrapp...,False,0,True,False,7033841630654549294,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 42, 'hashtagName': 'wrappinggifts', 'isCommerce...",https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,0,0,0,False,6875149358373176325,False,False,Margaret & Stetson,False,False,0,MS4wLjABAAAAdK-fCBiHfDnxlcqWI8k0t8JSvC2zZ2elAJFE2WLjwpFmLRrGIHCVo8_e6Ww...,False,"Mother-daughter duo\nAmazon finds, home organization, & more\nSHOP OUR ...",0,everythingenvy,False,True,We Wish You a Merry Christmas,Merry Christmas,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,29.0,6709421144174233602,False,We Wish You a Merry Christmas,121,4,630,17000,67,121,4,630,17000,0,67,0,1453478.0,"[{'Bitrate': 1453478, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5a1b8d9d39e...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,12,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,,normal,mp4,1024,7033841630654549294,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-12.9,0.89125,576,https://p19-sign.tiktokcdn-us.com/tos-useast5-p-0068-tx/

We'll then build the URL for the video and make sure to remove any duplicates.

In [111]:
# Use .insert to put the URL in the front so we can see it easily here
urls = 'https://tiktok.com/@' + videos_df['author_uniqueId'] + '/video/' + videos_df['video_id']
videos_df.insert(0, 'tiktok_url', urls)

# Drop duplicates based on tiktok_urls
videos_df = videos_df.drop_duplicates(subset=['tiktok_url'])
print(videos_df.shape[0], "total videos")

videos_df.head()

5193 total videos


,tiktok_url,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@everythingenvy/video/7033841630654549294,,[{'coverLarger': 'https://p16-amd-va.tiktokcdn.com/obj/musically-maliva...,False,[{'desc': 'How to make a handmade bow! #wrappinggifts #wrappingpresents...,1637693876,How to make a handmade bow! #wrappinggifts #wrappingpresents #giftwrapp...,False,0,True,False,7033841630654549294,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 42, 'hashtagName': 'wrappinggifts', 'isCommerce...",https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,https://p16-sign.tiktokcdn-us.com/tos-useast5-avt-0068-tx/42140f5821e99...,0,0,0,False,6875149358373176325,False,False,Margaret & Stetson,False,False,0,MS4wLjABAAAAdK-fCBiHfDnxlcqWI8k0t8JSvC2zZ2elAJFE2WLjwpFmLRrGIHCVo8_e6Ww...,False,"Mother-daughter duo\nAmazon finds, home organization, & more\nSHOP OUR ...",0,everythingenvy,False,True,We Wish You a Merry Christmas,Merry Christmas,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/02858d8a12b646579485...,29.0,6709421144174233602,False,We Wish You a Merry Christmas,121,4,630,17000,67,121,4,630,17000,0,67,0,1453478.0,"[{'Bitrate': 1453478, 'CodecType': 'h264', 'GearName': 'normal_540_0', ...",h264,https://p16-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/5a1b8d9d39e...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,12,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,,normal,mp4,1024,7033841630654549294,https://p19-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-1

Now we'll save them.

In [112]:
videos_df['author_nickname'].value_counts()

author_nickname
AMAZON FINDS          1255
Sarah Lindner         1144
Jared Andersen        1092
Margaret & Stetson     796
Sam Shan Shops 🦋       487
Anya                   264
Justice Buys           155
Name: count, dtype: int64

In [119]:
filtered_df = videos_df[videos_df['author_nickname'] == "AMAZON FINDS"]

In [120]:
filtered_df.to_csv("ariellebrimhall_videos.csv", index=False)

## Download videos

Time to use [yt-dlp](https://github.com/yt-dlp/yt-dlp) to actually download the videos! We'll start by opening the CSV file.

In [48]:
import pandas as pd

In [115]:
import os
import yt_dlp
from yt_dlp.utils import ExtractorError, DownloadError
from urllib.error import HTTPError

df = pd.read_csv("ariellebrimhall_videos.csv")

df.head()


,tiktok_url,AIGCDescription,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,itemCommentStatus,officalItem,originalItem,privateItem,secret,shareEnabled,stitchDisplay,stitchEnabled,textExtra,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_uniqueId,author_verified,item_control_can_repost,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_repostCount,statsV2_shareCount,video_VQScore,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,music_playUrl,diversificationId,video_subtitleInfos,playlistId,anchors,adLabelVersion,adAuthorization,isAd,item_control_can_comment,item_control_can_creator_redirect,item_control_can_music_redirect,item_control_can_share,itemMute,poi_address,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_typeCode,BAInfo,isECVideo,music_scheduleSearchTime,duetInfo_duetFromId,videoSuggestWordsList_video_suggest_words_struct,warnInfo,isPinnedItem,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title
0,https://tiktok.com/@ariellebrimhall/video/7125566784958106926,NaN,"[{'coverLarger': '', 'coverMedium': '', 'coverThumb': '', 'desc': '', '...",False,[{'desc': 'everything in my amazon storefront #amazondormfinds #backtos...,1659050303,everything in my amazon storefront #amazondormfinds #backtoschool #dorm...,False,0,True,False,7125566784958106926,0,False,False,False,False,True,0,True,"[{'awemeId': '', 'end': 51, 'hashtagName': 'amazondormfinds', 'isCommer...",https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,https://p19-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/73...,0,0,0,False,6907257263154971653,False,False,AMAZON FINDS,False,False,0,MS4wLjABAAAAL88tp6xEG4omf2EsYDEouVXDHi4kvlCKHaMB_O1MPUMSQWgMa7Srt57BtCt...,False,Amazon Influencer | IG @ariellebrimhall\nCLICK LINK TO SHOP FINDS 🛍,0,ariellebrimhall,False,True,NaN,TimTaj,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/adc796de52394e04ae2e...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/adc796de52394e04ae2e...,https://p16-sign-sg.tiktokcdn.com/tos-alisg-v-2774/adc796de52394e04ae2e...,135.0,6817342688401033218,False,Summer day,29,0,85,5796,10,29,0,85,5796,0,10,0.0,159659.0,"[{'Bitrate': 173371, 'CodecType': 'h265_hvc1', 'GearName': 'adapt_lower...",h264,https://p16-pu-sign-useast8.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/...,540p,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,16,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/833b23ebf15...,NaN,normal,mp4,1024,7125566784958106926,https://p19-sign.tiktokcdn-us.com/obj/tos-useast5-p-0068-tx/d2f437a5625...,https://v16-webapp-prime.us.tiktok.com/video/tos/useast5/tos-useast5-ve...,540p,normal,-11.6,1.00000,576,https://p19-pu-sign-useast8.tiktokcdn-us.com/to

In [116]:
df.shape

(1255, 129)

In [50]:
from tqdm import tqdm

In [117]:
import os
import yt_dlp
from tqdm import tqdm
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Assuming df is your DataFrame
# For example, you can define a dummy df like this for testing:
# df = pd.DataFrame({'tiktok_url': ['url1', 'url2', ...], 'author_uniqueId': ['author1', 'author2', ...]})

for i, row in tqdm(list(df.iterrows())[:300], desc="Downloading videos"):
    url = row['tiktok_url']

    ydl_opts = {
        "outtmpl": os.path.join('videos', row['author_uniqueId'], "%(id)s.%(ext)s"),
        "ignore_errors": True,
        "quiet": True,
    }

    print("Downloading", url)

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
        except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
            # Catch urllib and yt-dlp errors when video not found
            print(f"Encountered error {e} when attempting to download url: {url}")


ERROR: [TikTok] 7123985831936691499: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7123985831936691499: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@ariellebrimhall/video/7123985831936691499


ERROR: [TikTok] 7062449198142344495: Video not available, status code 100004; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7062449198142344495: Video not available, status code 100004; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@ariellebrimhall/video/7062449198142344495


ERROR: [TikTok] 7263515230687218986: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

Encountered error ERROR: [TikTok] 7263515230687218986: Unable to extract webpage video data; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U when attempting to download url: https://tiktok.com/@ariellebrimhall/video/7263515230687218986


In [118]:
all_videos = glob.glob("videos/thehouseofsequins/*.mp4", recursive=True)
len(all_videos)

297

In [15]:
# # In this case we're only downloading the first 40 using [:40]

# for i, row in list(df.iterrows())[:40]:    
#     url = row['tiktok_url']

#     ydl_opts = {
#         "outtmpl": os.path.join('videos', row['author_uniqueId'], "%(id)s.%(ext)s"),
#         "ignore_errors": True,
#         "quiet": True,
#     }

#     print("Downloading", url)

#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         try:
#             ydl.download([url])
#         except (HTTPError, TypeError, ExtractorError, DownloadError) as e:
#             # Catch urllib and yt-dlp errors when video not found
#             print(
#                 f"Encountered error {e} when attempting to download url: {url}"
#             )

## Get comments

Same process as above, but for `api/comment/list/`.

In [61]:
import glob

# Just grab the first one
filename = glob.glob("api/comment/list/*.json")[0]

In [62]:
with open(filename) as fp:
    content = json.load(fp)

content['data']['comments'][0]

{'author_pin': False,
 'aweme_id': '7375576798093970721',
 'cid': '7375577986069152558',
 'collect_stat': 0,
 'comment_language': 'en',
 'comment_post_item_ids': None,
 'create_time': 1717260598,
 'digg_count': 5805,
 'image_list': None,
 'is_author_digged': False,
 'label_list': None,
 'no_show': False,
 'reply_comment': None,
 'reply_comment_total': 210,
 'reply_id': '0',
 'reply_to_reply_id': '0',
 'share_info': {'acl': {'code': 0, 'extra': '{}'},
  'desc': 'Mr Trix2046のコメント: Who else came here because of the forgotten slide show',
  'title': 'I’m a little confused #learnfromkhaby #comedy #football ',
  'url': 'https://m.tiktok.com/v/7375576798093970721.html?_d=eei61e1lh2mjhc&comment_author_id=7155234282889085994&preview_pb=0&share_comment_id=7375577986069152558&share_item_id=7375576798093970721&sharer_language=ja-JP&source=h5_m&u_code=0'},
 'status': 1,
 'stick_position': 0,
 'text': 'Who else came here because of the forgotten slide show',
 'text_extra': [],
 'trans_btn_style': 1,

In [63]:
import glob

# Just grab the first one
filenames = glob.glob("api/comment/list/*.json")
filenames[:10]

['api/comment/list/967177398f4aec8405251869d168a09c.json',
 'api/comment/list/deebcb3b860201f03e24afa4f12fb963.json',
 'api/comment/list/04315bef36d60459455d0e7fb89f3cab.json',
 'api/comment/list/fefa787668ce6e96bf86e30338e82216.json',
 'api/comment/list/b001850da47215f00551d9e686153124.json',
 'api/comment/list/175a4aa2e2c7909bb7e01a4d4f051bdc.json',
 'api/comment/list/836c3bfb03e29d30164037e3f617f3ad.json',
 'api/comment/list/7b339f59ba8299f10b45b3c126b3865d.json',
 'api/comment/list/32bcaa0c116cd98eff05985266b1b2f7.json',
 'api/comment/list/475e74fd668d201a520cb7ebdbe10350.json']

In [64]:

rows = []

for filename in filenames:
    with open(filename) as fp:
        content = json.load(fp)
        rows.extend(content['data']['comments'])

len(rows)

562

In [65]:
import pandas as pd
pd.options.display.max_columns = None

# Use json_normalize bc see above
comments_df = pd.json_normalize(rows, sep='_')

comments_df.head(2)

,author_pin,aweme_id,cid,collect_stat,comment_language,comment_post_item_ids,create_time,digg_count,image_list,is_author_digged,label_list,no_show,reply_comment,reply_comment_total,reply_id,reply_to_reply_id,status,stick_position,text,text_extra,trans_btn_style,user_buried,user_digged,share_info_acl_code,share_info_acl_extra,share_info_desc,share_info_title,share_info_url,user_account_labels,user_ad_cover_url,user_advance_feature_item_order,user_advanced_feature_info,user_avatar_thumb_uri,user_avatar_thumb_url_list,user_avatar_thumb_url_prefix,user_bold_fields,user_can_message_follow_status_list,user_can_set_geofencing,user_cha_list,user_cover_url,user_custom_verify,user_enterprise_verify_reason,user_events,user_followers_detail,user_geofencing,user_homepage_bottom_toast,user_item_list,user_mutual_relation_avatars,user_need_points,user_nickname,user_platform_sync_info,user_relative_users,user_search_highlight,user_sec_uid,user_shield_edit_field_info,user_type_label,user_uid,user_unique_id,user_user_profile_guide,user_user_tags,user_white_cover_url,qrec_virtual_enable
0,False,7375576798093970721,7375577986069152558,0,en,None,1717260598,5805,None,False,None,False,None,210,0,0,1,0,Who else came here because of the forgotten slide show,[],1,False,0,0,{},Mr Trix2046のコメント: Who else came here because of the forgotten slide show,I’m a little confused #learnfromkhaby #comedy #football,https://m.tiktok.com/v/7375576798093970721.html?_d=eei61e1lh2mjhc&comme...,None,None,None,None,tos-useast5-avt-0068-tx/c1c7c6a5804a3617c7b3c0d4997ff671,[https://p16-pu-sign-useast8.tiktokcdn-us.com/tos-useast5-avt-0068-tx/c...,None,None,None,None,None,None,,,None,None,None,None,None,None,None,Mr Trix2046,None,None,None,MS4wLjABAAAAV1BjJt9kH6Yv-KFZ44vPRLVqD6rJhJXDAqjHBrTiUu1W6end7wrHACwBgUx...,None,None,7155234282889085994,travipaty0,None,None,None,NaN
1,False,7375576798093970721,7375591554548761362,0,en,None,1717263765,30,None,False,None,False,None,2,0,0,1,0,rate my edit🙏,[],1,False,0,0,{},𝖈𝖆𝖒𝖎_のコメント: rate my edit🙏,I’m a little confused #learnfromkhaby #comedy #football,https://m.tiktok.com/v/7375576798093970721.html?_d=eei61e1lh2mjhc&comme...,None,None,None,None,tos-alisg-avt-0068/2fb20588abc06046e7240672dc0a388a,[https://p16-sign-sg.tiktokcdn.com/aweme/100x100/tos-alisg-avt-0068/2fb...,None,None,None,None,None,None,,,None,None,None,None,None,None,None,𝖈𝖆𝖒𝖎_,None,None,None,MS4wLjABAAAAyBFuGqxSua1rJOkdaTZc6j8rcPzqvIGMEYpOz0cQifsHI9SOZzwxwSQqNGK...,None,None,7307970066262574098,its_camixpp,None,None,None,NaN


In [66]:
# Remove duplicates
comments_df = comments_df.drop_duplicates(subset=['aweme_id', 'cid'])
print(comments_df.shape[0])

331


In [15]:
comments_df.to_csv("comments.csv", index=False)

## Searching and filtering

Video `id` field matches comment `aweme_id` field. You can take the first one above - I'm seeing `6892488417905577222` - and use it to filter (or even join)

In [73]:
# Search videos by video id
videos_df[videos_df.id == '6892488417905577222']

,BAInfo,adAuthorization,adLabelVersion,aigcLabelType,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,itemMute,officalItem,originalItem,playlistId,privateItem,secret,shareEnabled,showNotPass,stitchDisplay,stitchEnabled,textExtra,vl1,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_ttSeller,author_uniqueId,author_verified,authorStats_diggCount,authorStats_followerCount,authorStats_followingCount,authorStats_friendCount,authorStats_heart,authorStats_heartCount,authorStats_videoCount,duetInfo_duetFromId,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_playUrl,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_shareCount,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_reflowCover,video_shareCover,video_size,video_subtitleInfos,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,diversificationId,anchors,warnInfo,stickersOnItem,effectStickers,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title,poi_address,poi_category,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiId,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_type,poi_typeCode,maskType
3,,False,0,0,[{'coverLarger': 'https://p16-amd-va.tiktokcdn...,False,[{'desc': '#greenscreen Biden funna make Ameri...,1604782549,#greenscreen Biden funna make America great ag...,False,0,True,False,6892488417905577222,False,0,True,False,False,,False,False,True,False,0,True,"[{'awemeId': '', 'end': 12, 'hashtagId': '8018...",False,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p16-pu-sign-useast8.tiktokcdn-us.com/t...,https://p16-pu-sign-useast8.tiktokcdn-us.com/t...,0,3,0,False,6681843491281126406,False,False,Jay and Nia,True,False,0,MS4wLjABAAAAxtJs-CKw7wXRqmf-9AxsMda78a0xCiPq9o...,False,Followers Of Jesus 🫶🏾\n📧: Jay.nia4l@gmail.com\...,0,False,jayandnia,False,3169,6600000,5,0,135900000,135900000,1776,0,,Bella,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,10,6885917439331240710,True,,original sound,662,273,28200,368000,346,662,273,28248,367966,346,1227352.0,"[{'Bitrate': 1227352, 'CodecType': 'h264', 'Ge...",h264,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,540p,https://v16-webapp-prime.us.tiktok.com/video/t...,10,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,,normal,mp4,1024,6892488417905577222,https://p16-sign-va.tiktokcdn.com/obj/tos-mali...,https://v16-webapp-prime.us.tiktok.com/video/t...,540p,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,"[, https://p16-sign-va.tiktokcdn.com/tos-maliv...",1602922.0,NaN,normal,NaN,NaN,576,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,https://p16-sign-va.tiktokcdn.com/tos-maliva-p...,10024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Search comments by user id
comments_df[comments_df.user_uid == '6861221316131128325']

,author_pin,aweme_id,cid,collect_stat,comment_language,comment_post_item_ids,create_time,digg_count,image_list,is_author_digged,label_list,no_show,reply_comment,reply_comment_total,reply_id,reply_to_reply_id,status,stick_position,text,text_extra,trans_btn_style,user_buried,user_digged,share_info_acl_code,share_info_acl_extra,share_info_desc,share_info_title,share_info_url,user_account_labels,user_ad_cover_url,user_advance_feature_item_order,user_advanced_feature_info,user_avatar_thumb_uri,user_avatar_thumb_url_list,user_avatar_thumb_url_prefix,user_bold_fields,user_can_message_follow_status_list,user_can_set_geofencing,user_cha_list,user_cover_url,user_custom_verify,user_enterprise_verify_reason,user_events,user_followers_detail,user_geofencing,user_homepage_bottom_toast,user_item_list,user_mutual_relation_avatars,user_need_points,user_nickname,user_platform_sync_info,user_relative_users,user_search_highlight,user_sec_uid,user_shield_edit_field_info,user_type_label,user_uid,user_unique_id,user_user_profile_guide,user_user_tags,user_white_cover_url,label_text,label_type
0,False,6892488417905577222,6893132880093085701,0,,None,1604932581,2,None,False,None,False,None,1,0,0,1,0,Did u realized u have a blue oufit on fire bid...,[],0,False,0,0,{},TheOnly.Nieのコメント: Did u realized u have a blue...,#greenscreen Biden funna make America great ag...,https://m.tiktok.com/v/6892488417905577222.htm...,None,None,None,None,tos-maliva-avt-0068/5504628a898bba6104b320e944...,[https://p16-sign-va.tiktokcdn.com/tos-maliva-...,None,None,None,None,None,None,,,None,None,None,None,None,None,None,TheOnly.Nie,None,None,None,MS4wLjABAAAADN5RqCjUXdp2VBHLV3zgEJxVMyheVfpZDE...,None,None,6861221316131128325,naellefrancillon6,None,None,None,NaN,NaN
162,False,6892488417905577222,6893132880093085701,0,,None,1604932581,2,None,False,None,False,None,1,0,0,1,0,Did u realized u have a blue oufit on fire bid...,[],0,False,0,0,{},TheOnly.Nieのコメント: Did u realized u have a blue...,#greenscreen Biden funna make America great ag...,https://m.tiktok.com/v/6892488417905577222.htm...,None,None,None,None,tos-maliva-avt-0068/5504628a898bba6104b320e944...,[https://p16-sign-va.tiktokcdn.com/tos-maliva-...,None,None,None,None,None,None,,,None,None,None,None,None,None,None,TheOnly.Nie,None,None,None,MS4wLjABAAAADN5RqCjUXdp2VBHLV3zgEJxVMyheVfpZDE...,None,None,6861221316131128325,naellefrancillon6,None,None,None,NaN,NaN


In [75]:
# Search videos by user id
videos_df[videos_df.author_id == '6886175799079633925']

,BAInfo,adAuthorization,adLabelVersion,aigcLabelType,challenges,collected,contents,createTime,desc,digged,duetDisplay,duetEnabled,forFriend,id,isAd,itemCommentStatus,itemMute,officalItem,originalItem,playlistId,privateItem,secret,shareEnabled,showNotPass,stitchDisplay,stitchEnabled,textExtra,vl1,author_avatarLarger,author_avatarMedium,author_avatarThumb,author_commentSetting,author_downloadSetting,author_duetSetting,author_ftc,author_id,author_isADVirtual,author_isEmbedBanned,author_nickname,author_openFavorite,author_privateAccount,author_relation,author_secUid,author_secret,author_signature,author_stitchSetting,author_ttSeller,author_uniqueId,author_verified,authorStats_diggCount,authorStats_followerCount,authorStats_followingCount,authorStats_friendCount,authorStats_heart,authorStats_heartCount,authorStats_videoCount,duetInfo_duetFromId,music_album,music_authorName,music_coverLarge,music_coverMedium,music_coverThumb,music_duration,music_id,music_original,music_playUrl,music_title,stats_collectCount,stats_commentCount,stats_diggCount,stats_playCount,stats_shareCount,statsV2_collectCount,statsV2_commentCount,statsV2_diggCount,statsV2_playCount,statsV2_shareCount,video_bitrate,video_bitrateInfo,video_codecType,video_cover,video_definition,video_downloadAddr,video_duration,video_dynamicCover,video_encodeUserTag,video_encodedType,video_format,video_height,video_id,video_originCover,video_playAddr,video_ratio,video_reflowCover,video_shareCover,video_size,video_subtitleInfos,video_videoQuality,video_volumeInfo_Loudness,video_volumeInfo_Peak,video_width,video_zoomCover_240,video_zoomCover_480,video_zoomCover_720,video_zoomCover_960,diversificationId,anchors,warnInfo,stickersOnItem,effectStickers,imagePost_cover_imageHeight,imagePost_cover_imageURL_urlList,imagePost_cover_imageWidth,imagePost_images,imagePost_shareCover_imageHeight,imagePost_shareCover_imageURL_urlList,imagePost_shareCover_imageWidth,imagePost_title,poi_address,poi_category,poi_city,poi_cityCode,poi_country,poi_countryCode,poi_fatherPoiId,poi_fatherPoiName,poi_id,poi_name,poi_province,poi_ttTypeCode,poi_ttTypeNameMedium,poi_ttTypeNameSuper,poi_ttTypeNameTiny,poi_type,poi_typeCode,maskType
0,,False,0,0,"[{'coverLarger': '', 'coverMedium': '', 'cover...",False,[{'desc': 'Replying to @chiyufu #twitter #Intu...,1669022418,Replying to @chiyufu #twitter #IntuitTouchdown...,False,0,True,False,7168396273824091438,False,0,False,False,False,7121046092082613038,False,False,True,False,0,True,"[{'awemeId': '', 'end': 20, 'hashtagId': '', '...",False,https://p16-pu-sign-useast8.tiktokcdn-us.com/t...,https://p16-pu-sign-useast8.tiktokcdn-us.com/t...,https://p19-pu-sign-useast8.tiktokcdn-us.com/t...,0,0,3,False,6886175799079633925,False,False,Patriot🇺🇸🇺🇸,False,False,0,MS4wLjABAAAArujQcmgMwacdXeGwSNgQubXqPyKPXmnpAP...,False,40K?🚀\nI POST EDITS! \n‼️SPAM=BLOCK‼️\n(I POST...,3,False,americanboyforpresident,False,21000,36100,914,0,2000000,2000000,711,0,Override (Slowed + Reverb),KSLV Noh,https://p16-sg.tiktokcdn.com/aweme/720x720/tos...,https://p16-sg.tiktokcdn.com/aweme/200x200/tos...,https://p16-sg.tiktokcdn.com/aweme/100x100/tos...,60,7047249385885566977,False,https://sf16-ies-music-sg.tiktokcdn.com/obj/to...,Override - Slowed + Reverb,322,10,1427,15800,32,322,10,1427,15801,32,803605.0,"[{'Bitrate': 803605, 'CodecType': 'h264', 'Gea...",h264,https://p16-sign.tiktokcdn-us.com/obj/tos-usea...,540p,https://v16-webapp-prime.us.tiktok.com/video/t...,19,https://p16-sign.tiktokcdn-us.com/obj/tos-usea...,,normal,mp4,576,7168396273824091438,https://p16-sign.tiktokcdn-us.com/obj/tos-usea...,https://v16-webapp-prime.us.tiktok.com/video/t...,540p,https://p19-sign.tiktokcdn-us.com/tos-useast5-...,"[, https://p19-sign.tiktokcdn-us.com/tos-useas...",1938195.0,"[{'Format': 'webvtt', 'LanguageCodeName': 'ind...",normal,-5.4,1.0,576,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,https://p16-sign.tik